# Scale Scikit-Learn with Dask and Joblib

<img src="https://raw.githubusercontent.com/scikit-learn/scikit-learn/master/doc/logos/scikit-learn-logo-notext.png"/> <img src="http://joblib.readthedocs.io/en/latest/_static/joblib_logo.svg" width="20%"/> 

Many Scikit-Learn operations already support parallelism with [joblib](http://joblib.readthedocs.io/) for single-machine parallelism. This lets you train most estimators (anything that accepts an `n_jobs` parameter) using all the cores of your laptop or workstation.

Dask can scale these operations by replacing Joblib's single-machine system with a distributed cluster.  This lets you train those estimators using all the cores of your *cluster*, by changing one line of code. 

This is most useful for training large models on medium-sized datasets. You may have a large model when searching over many hyper-parameters, or when using an ensemble method with many individual estimators. For too small datasets, training times will typically be small enough that cluster-wide parallelism isn't helpful. For too large datasets (larger than a single machine's memory), the scikit-learn estimators may not be able to cope (see below).

In [ ]:
from dask.distributed import Client, progress
client = Client()
client.restart()

## Create a Dataset, Model, and Grid Search

In [ ]:
# import dask_ml.joblib  # register the distriubted backend
from sklearn.datasets import make_classification
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
import pandas as pd

We'll use scikit-learn to create a pair of small random arrays, one for the features `X`, and one for the target `y`.

In [ ]:
X, y = make_classification(n_samples=1000, random_state=0)
X[:5]

We'll fit a [Support Vector Classifier](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html), using [grid search](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to find the best value of the $C$ hyperparameter.

In [ ]:
param_grid = {"C": np.logspace(-3, 1, 30),
              "gamma": [0.05, 0.5, 2],
              "kernel": ['rbf', 'poly', 'sigmoid'],
              "shrinking": [True, False]}

grid_search = GridSearchCV(SVC(gamma='auto', random_state=0, probability=True),
                           param_grid=param_grid,
                           return_train_score=False,
                           iid=True,
                           cv=3,
                           n_jobs=-1)

## Train on a Cluster

To fit that normally, we'd call

```python
grid_search.fit(X, y)
```

To fit it using the cluster, we just need to use a context manager provided by joblib.
We'll pre-scatter the data to each worker, which can help with performance.

In [ ]:
%%time

import joblib

with joblib.parallel_backend('dask', scatter=[X, y]):
    grid_search.fit(X, y)

We fit 1620 different models, one for each hyper-parameter and CV-split combination. The training was distributed across the cluster. At this point, we have a regular scikit-learn model, which can be used for prediction, scoring, etc.

In [ ]:
res = pd.DataFrame(grid_search.cv_results_)
res.head()

## The result is just a Scikit-Learn object

Dask didn't replace Scikti-Learn, so everything is the same as you are accustomed to.

In [ ]:
grid_search.score(X, y)

For more on training scikit-learn models with distributed joblib, see the [dask-ml documentation](http://dask-ml.readthedocs.io/en/latest/joblib.html).